# Play Against AlphaZero

This notebook allows you to play interactively against the trained AlphaZero model.

In [4]:
import sys
import os
from pathlib import Path
import json

import torch
from rgi.rgizero.games import game_registry
from rgi.rgizero.experiment import ExperimentRunner, ExperimentConfig
from rgi.rgizero.players.alphazero import AlphazeroPlayer
from rgi.rgizero.players.human_player import HumanPlayer
from rgi.rgizero.evaluators import ActionHistoryTransformerEvaluator

import notebook_utils
from notebook_utils import reload_local_modules


In [5]:
# Configuration
EXPERIMENT_NAME = "smoketest-e2e-v7"
GEN_ID = 10 # Generation to load (or None for latest? Notebook usually specifies)
GAME_NAME = "connect4"

DEVICE = notebook_utils.detect_device()

Detected device: mps


In [6]:
# Load Experiment
base_dir = Path.cwd().parent / 'experiments'
exp_dir = base_dir / EXPERIMENT_NAME
config = ExperimentConfig.from_json(json.load(open(exp_dir / "config.json")))
runner = ExperimentRunner(config, base_dir=base_dir)

# Load Model
model = runner.load_model(GEN_ID)
model.to(DEVICE)
model.eval()

print(f"Loaded model from generation {GEN_ID}")

Loaded model from generation 10


In [ ]:
# Setup Players
game = game_registry.create_game(GAME_NAME)
evaluator = ActionHistoryTransformerEvaluator(model, device=DEVICE, block_size=runner.n_max_context, vocab=runner.action_vocab)

# Adjust simulations or temperature as needed
ai_player = AlphazeroPlayer(game, evaluator, simulations=200, temperature=0.0)
human_player = HumanPlayer(game)

# Choose side: Human vs AI (Player 1 vs Player 2)
players = [human_player, ai_player] 
# players = [ai_player, human_player] # Uncomment to let AI go first

print("Players set up. Run the next cell to play!")

TypeError: ActionHistoryTransformerEvaluator.__init__() missing 1 required positional argument: 'vocab'

In [ ]:
state = game.initial_state()

while not game.is_terminal(state):
    current_p_idx = game.current_player_id(state) - 1
    player = players[current_p_idx]
    
    # Human player handles printing board inside _select_action_from_user
    # But for AI turn, we might want to print it too to see what's happening
    if player != human_player:
        print(f"\nAI (Player {current_p_idx+1}) thinking...")
        # print(game.pretty_str(state))
    
    result = player.select_action(state)
    action = result.action
    
    if player != human_player:
        print(f"AI chose: {action}")
    
    state = game.next_state(state, action)

print("\nGame Over!")
print(game.pretty_str(state))
print(f"Winner: {game.get_rewards(state)}")

In [ ]:
import json